# <centered> Week 10

## Part 1: Keras NN

Use the Keras library to fit a neural network to the housing dataset to predict whether the house sold above or below the median.  You just have to fit the model and find estimate the test accuracy (you can just do a 90/10 split and not run full cross-validation).

In [ ]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout 
import keras
from keras import layers

from sklearn.metrics import accuracy_score 
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.compose import make_column_selector
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder

import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('house_SalePrice.csv')
df['above_median'] = df['SalePrice'].apply(lambda x: 1 if x > np.median(df['SalePrice']) else 0)
df.drop(['SalePrice'], axis = 1, inplace=True)

In [ ]:
X_train , X_test,y_train, y_test = train_test_split(df.drop(['above_median'], axis = 1), 
                                                    df['above_median'],
                                                     test_size=0.1, random_state=42)

### Data Preprocessing

In [ ]:
col_trans = make_column_transformer((OneHotEncoder(handle_unknown = 'ignore', 
                                                   drop = 'first'), 
                                     make_column_selector(dtype_include = object)),
                                    (StandardScaler(), 
                                     make_column_selector(dtype_include=np.number)),
                                  remainder = 'passthrough')

imputer = SimpleImputer(strategy='most_frequent')


### Preprocessing

In [ ]:
col_trans.fit(X_train)
X_train_trans = col_trans.transform(X_train)
imputer.fit(X_train_trans)
X_train_trans = imputer.transform(X_train_trans)
y_train_trans = y_train.values

In [ ]:
X_train_trans.shape

(1440, 84)

In [ ]:
y_train_trans.shape

(1440,)

### Keras Model

In [ ]:
model = Sequential()
model.add(Dense(128, input_dim=X_train_trans.shape[1]))
model.add(Activation('relu'))
model.add(Dropout(0.15))
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.15))
model.add(Dense(1))
model.add(Activation('sigmoid'))
model.compile(
              loss=keras.losses.BinaryCrossentropy(),
              metrics=[keras.metrics.BinaryAccuracy(),
                       keras.metrics.FalseNegatives()])
model.fit(X_train_trans, y_train_trans, epochs = 100, batch_size = 16)

Epoch 1/100
90/90 [==============================] - 8s 5ms/step - loss: 0.3520 - binary_accuracy: 0.8597 - false_negatives: 105.0000
Epoch 2/100
90/90 [==============================] - 0s 5ms/step - loss: 0.2571 - binary_accuracy: 0.8965 - false_negatives: 74.0000
Epoch 3/100
90/90 [==============================] - 0s 5ms/step - loss: 0.2435 - binary_accuracy: 0.9007 - false_negatives: 72.0000
Epoch 4/100
90/90 [==============================] - 0s 5ms/step - loss: 0.2203 - binary_accuracy: 0.9069 - false_negatives: 72.0000
Epoch 5/100
90/90 [==============================] - 0s 5ms/step - loss: 0.2183 - binary_accuracy: 0.9090 - false_negatives: 66.0000
Epoch 6/100
90/90 [==============================] - 0s 5ms/step - loss: 0.2023 - binary_accuracy: 0.9208 - false_negatives: 59.0000
Epoch 7/100
90/90 [==============================] - 0s 5ms/step - loss: 0.1963 - binary_accuracy: 0.9222 - false_negatives: 56.0000
Epoch 8/100
90/90 [==============================] - 0s 5ms/step - l

### Transform the test data

In [ ]:
X_test_trans = col_trans.transform(X_test)
imputer.fit(X_test_trans)
X_test_trans = imputer.transform(X_test_trans)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_encoders.py:174: UserWarning: Found unknown categories in columns [1] during transform. These unknown categories will be encoded as all zeros
  UserWarning,


### Get the predictions and out of sample accuracy

In [ ]:
preds = model.predict(X_test_trans)

In [ ]:
preds_keras = list(np.argmax(preds, axis = 1))

In [ ]:
model.evaluate(X_test_trans, y_test.values)

6/6 [==============================] - 1s 62ms/step - loss: 2.0385 - binary_accuracy: 0.8882 - false_negatives: 13.0000


[2.0385117530822754, 0.888198733329773, 13.0]

This model got a very high accuracy score while fitting on the training data (99.65%). However, when I evaluate the model with the testing data, I get a lower accuracy score (88.8%) than the models I ran in the previous homework: the random forest (95 %), multinomial logit (91,2 %), and the MLP classifier (96%).